In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score
from catboost import CatBoostClassifier, Pool
import pickle
import warnings
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 7)
plt.rcParams['font.size'] = 12

## 1. Загрузка и обзор данных

In [ ]:
df_trans = pd.read_csv("../docs/транзакции в Мобильном интернет Банкинге.csv", 
                       sep=";", encoding="cp1251", header=1)
df_behavior = pd.read_csv("../docs/поведенческие паттерны клиентов.csv", 
                          sep=";", encoding="cp1251", header=1)

print("ОБЗОР ДАННЫХ")
print("="*60)
print(f"Транзакции: {df_trans.shape[0]:,} записей, {df_trans.shape[1]} колонок")
print(f"Поведенческие паттерны: {df_behavior.shape[0]:,} записей, {df_behavior.shape[1]} колонок")
print(f"\nРаспределение классов:")
print(f"   Non-Fraud: {(df_trans['target']==0).sum():,} ({(df_trans['target']==0).mean()*100:.2f}%)")
print(f"   Fraud:     {(df_trans['target']==1).sum():,} ({(df_trans['target']==1).mean()*100:.2f}%)")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sizes = [df_trans['target'].sum(), (df_trans['target']==0).sum()]
labels = [f'Fraud\n({sizes[0]})', f'Non-Fraud\n({sizes[1]:,})']
colors = ['#e74c3c', '#2ecc71']
axes[0].pie(sizes, labels=labels, colors=colors, autopct='%1.2f%%', startangle=90,
           explode=(0.1, 0), textprops={'fontsize': 12})
axes[0].set_title('Распределение классов', fontsize=14, fontweight='bold')

fraud_per_client = df_trans[df_trans['target']==1].groupby('cst_dim_id').size()
axes[1].hist(fraud_per_client, bins=20, color='#e74c3c', edgecolor='black')
axes[1].set_xlabel('Количество fraud транзакций')
axes[1].set_ylabel('Количество клиентов')
axes[1].set_title('Распределение fraud по клиентам', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 2. Анализ поведенческих паттернов

In [ ]:
df_behavior['transdate'] = pd.to_datetime(df_behavior['transdate'].str.strip("'"))
df_trans['transdate'] = pd.to_datetime(df_trans['transdate'].str.strip("'"))
df_behavior['cst_dim_id'] = pd.to_numeric(df_behavior['cst_dim_id'], errors='coerce').fillna(0).astype(int).astype(str)
df_trans['cst_dim_id'] = pd.to_numeric(df_trans['cst_dim_id'], errors='coerce').fillna(0).astype(int).astype(str)

df_merged = df_trans.merge(df_behavior, on=['cst_dim_id', 'transdate'], how='left')
fraud_data = df_merged[df_merged['target'] == 1]
non_fraud_data = df_merged[df_merged['target'] == 0]

print(f"✅ Merged dataset: {df_merged.shape}")
print(f"   Fraud: {len(fraud_data)}, Non-Fraud: {len(non_fraud_data)}")

In [ ]:
key_features = ['logins_last_7_days', 'logins_last_30_days', 'avg_login_interval_30d', 
                'freq_change_7d_vs_mean', 'burstiness_login_interval']

print("FRAUD vs NON-FRAUD: Ключевые метрики")
print("="*70)
print(f"{'Метрика':<30} {'Fraud':<15} {'Non-Fraud':<15} {'Diff %':<10}")
print("-"*70)

for col in key_features:
    if col in df_merged.columns:
        f_mean = pd.to_numeric(fraud_data[col], errors='coerce').mean()
        nf_mean = pd.to_numeric(non_fraud_data[col], errors='coerce').mean()
        diff = ((f_mean - nf_mean) / abs(nf_mean) * 100) if nf_mean != 0 else 0
        indicator = "!!!" if abs(diff) > 20 else "  "
        print(f"{indicator} {col:<28} {f_mean:<15.2f} {nf_mean:<15.2f} {diff:>+.1f}%")

## 3. Two-Stage Detection Results

In [ ]:
results = pd.read_csv("../docs/two_stage_detection_results.csv")

total = len(results)
fraud_total = results['target'].sum()
scorecard_approved = results[results['detection_stage'] == 'scorecard']
ml_checked = results[results['detection_stage'] == 'ml_model']

fraud_missed_scorecard = len(scorecard_approved[scorecard_approved['target'] == 1])
fraud_in_ml = len(ml_checked[ml_checked['target'] == 1])
fraud_detected = len(ml_checked[(ml_checked['target'] == 1) & (ml_checked['fraud_prediction'] == 1)])
fraud_missed_ml = len(ml_checked[(ml_checked['target'] == 1) & (ml_checked['fraud_prediction'] == 0)])

print("TWO-STAGE DETECTION RESULTS")
print("="*60)
print(f"\nSTAGE 1: Scorecard Filter")
print(f"   Всего транзакций: {total:,}")
print(f"   Auto-approved: {len(scorecard_approved):,} ({len(scorecard_approved)/total*100:.1f}%)")
print(f"   Sent to ML: {len(ml_checked):,} ({len(ml_checked)/total*100:.1f}%)")

print(f"\nSTAGE 2: ML Model (CatBoost)")
print(f"   Fraud в ML: {fraud_in_ml} из {fraud_total}")
print(f"   Detected: {fraud_detected}")
print(f"   Missed: {fraud_missed_ml}")

print(f"\nИТОГОВЫЙ RECALL: {fraud_detected}/{fraud_total} = {fraud_detected/fraud_total*100:.1f}%")
print(f"   Пропущено Scorecard: {fraud_missed_scorecard}")
print(f"   Пропущено ML: {fraud_missed_ml}")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

auto_pct = len(scorecard_approved)/total*100
ml_pct = len(ml_checked)/total*100
axes[0].pie([auto_pct, ml_pct], labels=[f'Auto-Approve\n({auto_pct:.1f}%)', f'ML Check\n({ml_pct:.1f}%)'], 
           colors=['#2ecc71', '#e74c3c'], autopct='%1.1f%%', startangle=90, explode=(0.05, 0))
axes[0].set_title('Распределение нагрузки', fontsize=14, fontweight='bold')

recalls = [52.7, fraud_detected/fraud_total*100]  # CV recall vs Two-Stage
bars = axes[1].bar(['Pure ML\n(CV)', 'Two-Stage'], recalls, color=['#3498db', '#2ecc71'], edgecolor='black')
axes[1].set_ylabel('Recall (%)')
axes[1].set_title('Улучшение Recall', fontsize=14, fontweight='bold')
axes[1].set_ylim(0, 100)
for bar, val in zip(bars, recalls):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, f'{val:.1f}%', ha='center', fontweight='bold')

categories = ['Detected', 'Missed\nScorecard', 'Missed\nML']
values = [fraud_detected, fraud_missed_scorecard, fraud_missed_ml]
colors = ['#2ecc71', '#e74c3c', '#f39c12']
bars = axes[2].bar(categories, values, color=colors, edgecolor='black')
axes[2].set_ylabel('Количество')
axes[2].set_title(f'Fraud Detection (всего: {fraud_total})', fontsize=14, fontweight='bold')
for bar, val in zip(bars, values):
    axes[2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, str(val), ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('../docs/two_stage_results.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. 🎯 Feature Importance

In [ ]:
model = CatBoostClassifier()
model.load_model('../models/catboost_fraud_model.cbm')

with open('../models/feature_names.pkl', 'rb') as f:
    feature_names = pickle.load(f)

importance = model.get_feature_importance()
feat_imp = pd.DataFrame({'feature': feature_names, 'importance': importance})
feat_imp = feat_imp.sort_values('importance', ascending=False)

plt.figure(figsize=(12, 8))
top_15 = feat_imp.head(15)
colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, 15))[::-1]
plt.barh(top_15['feature'], top_15['importance'], color=colors)
plt.xlabel('Importance')
plt.title('Feature Importance - Top 15', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('../docs/feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nTOP 5 FEATURES:")
for i, row in feat_imp.head(5).iterrows():
    print(f"   {row['feature']}: {row['importance']:.2f}%")

## 5. 📈 Model Stability (Cross-Validation)

In [ ]:
# Cross-Validation
X = results[feature_names].copy()
y = results['target'].values

cat_features = ['direction', 'last_phone_model', 'last_os_ver']
cat_indices = [feature_names.index(f) for f in cat_features if f in feature_names]

for i, col in enumerate(X.columns):
    if col in cat_features:
        X[col] = X[col].fillna('unknown').astype(str)
    else:
        X[col] = pd.to_numeric(X[col], errors='coerce').fillna(0)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
metrics = {'auc': [], 'precision': [], 'recall': [], 'f1': []}

print("📊 5-FOLD CROSS-VALIDATION")
print("="*60)

for fold, (train_idx, val_idx) in enumerate(cv.split(X, y), 1):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    model_cv = CatBoostClassifier(iterations=500, learning_rate=0.05, depth=6,
                                   cat_features=cat_indices, verbose=0, random_seed=42)
    model_cv.fit(X_train, y_train)
    
    y_proba = model_cv.predict_proba(X_val)[:, 1]
    y_pred = model_cv.predict(X_val)
    
    metrics['auc'].append(roc_auc_score(y_val, y_proba))
    metrics['precision'].append(precision_score(y_val, y_pred))
    metrics['recall'].append(recall_score(y_val, y_pred))
    metrics['f1'].append(f1_score(y_val, y_pred))
    
    print(f"Fold {fold}: AUC={metrics['auc'][-1]:.4f}, Precision={metrics['precision'][-1]:.4f}, "
          f"Recall={metrics['recall'][-1]:.4f}, F1={metrics['f1'][-1]:.4f}")

print("\n" + "="*60)
print("📊 СРЕДНИЕ ЗНАЧЕНИЯ (± std):")
for m in ['auc', 'precision', 'recall', 'f1']:
    print(f"   {m.upper()}: {np.mean(metrics[m]):.4f} ± {np.std(metrics[m]):.4f}")

In [ ]:
# Визуализация стабильности
fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(4)
means = [np.mean(metrics[m]) for m in ['auc', 'precision', 'recall', 'f1']]
stds = [np.std(metrics[m]) for m in ['auc', 'precision', 'recall', 'f1']]

bars = ax.bar(x, means, yerr=stds, capsize=5, color=['#2ecc71', '#3498db', '#e74c3c', '#9b59b6'])
ax.set_ylabel('Score')
ax.set_title('Model Stability: 5-Fold Cross-Validation', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(['AUC', 'Precision', 'Recall', 'F1'])
ax.set_ylim(0, 1.1)

for bar, mean in zip(bars, means):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05, f'{mean:.3f}', 
            ha='center', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('../docs/cv_stability.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. 🏆 Итоговая сводка

### Архитектура Two-Stage Fraud Detection

| Stage | Компонент | Описание |
|-------|-----------|----------|
| 1 | **Scorecard** | 13 правил на основе поведенческих паттернов |
| 2 | **CatBoost ML** | 36 фичей, AUC 0.99 |

### Ключевые результаты

| Метрика | Значение |
|---------|----------|
| ROC-AUC | 0.992 ± 0.003 |
| Precision | 0.944 ± 0.051 |
| Recall (Two-Stage) | 82.4% |
| Ускорение | 55% авто-одобрено |

### Бизнес-ценность

1. **Ускорение обработки**: 55% транзакций одобряются мгновенно
2. **Two-Stage улучшает recall**: +56% по сравнению с чистым ML
3. **Высокая precision**: 94% — минимум ложных срабатываний
4. **Стабильность**: CV показывает ±0.003 разброс AUC